<h1 style="align: center;font-size: 18pt;"> Sampling </h1>
<hr style="height:2.5px">
<h2 style="align: center;font-size: 12pt;"> Summary:  In this example, we are going to show how to set up a "master" script to set up BICePs samplings. You are welcome to take this as a template for you and make necessary modification to get it work for you. All input files used in this tutorial are prepared from [this tutorial](https://biceps.readthedocs.io/en/latest/tutorials/Preparation/Preparation.html).

<hr style="height:2.5px">



<hr style="height:2.5px">
<h2 style="align: center;font-size: 12pt;"> Summary: In our previous work (DOI: <a href="https://onlinelibrary-wiley-com.libproxy.temple.edu/doi/pdf/10.1002/jcc.23738">10.1002/jcc.23738</a>), we determine solution-state conformational populations of the 14-membered macrocycle cineromycin B, using a combination of previously published sparse Nuclear Magnetic Resonance (NMR) observables and replica exchange molecular dynamic/Quantum mehcanics (QM)-refined conformational ensembles. Cineromycin B is a 14-membered macrolide antibiotic that has become increasingly known for having activity against methicillin-resistant Staphylococcus Aureus (MRSA). In this example, we show how to calculate the consistency of computational modeling with experiment, and the relative importance of reference potentials and other model parameters. </h2>
<hr style="height:2.5px">

First, we need to import source code for BICePs classes. More information of these classes can be found [here](https://biceps.readthedocs.io/en/latest/workflow.html).

In [2]:
# import source code
import sys, os, glob, cPickle
import numpy as np
import biceps

Now we need to specify input files and output files folder.

In [3]:
############ Initialization #############
# Specify necessary argument values

# REQUIRED: specify number of states
states=100

# REQUIRED: specify directory of input data (BICePs readable format)
dataFiles = 'noe_J'   

# REQUIRED: sort data and figure out what experimental restraints are included for each state
data = sort_data(dataFiles)

# REQUIRED: energy file name of each state (computational prior distribution)

energies_filename = 'energy.dat'
energies = loadtxt(energies_filename)
energies -= energies.min()  # set ground state to zero, just in case

# REQUIRED: specify outcome directory of BICePs sampling
outdir = 'results_ref_normal'
# Make a new directory if we have to
if not os.path.exists(outdir):
    os.mkdir(outdir)


Next, we need to specify number of steps for BICePs sampling. We recommend to run at least 1M steps for converged Monte Carlo samplings. In BICePs, we also prepare functions of checking convergence of Monte Carlo. More information can be found [here].

In [4]:
# REQUIRED: number of MCMC steps for each lambda
nsteps = 1000000

We need to specify what lambda values we want to use in BICePs samplings. Briefly, lambda values are similar to the parameters used in free energy perturbation (FEP) and has effect on the BICePs score. The lambda values represent how much prior information from computational modeling is included in BICePs sampling (1.0 means all, 0.0 means none). As we explained in [this work](https://pubs.acs.org/doi/10.1021/acs.jpcb.7b11871), one can consider BICePs score as the relative free energy change between different models. More lambda values will increase the samplings for [multistate Bennett acceptance ratio (MBAR)](http://www.alchemistry.org/wiki/Multistate_Bennett_Acceptance_Ratio) predictions in free energy change and populations. However more lambda values also will slow down the whole process of BICePs (as more samplings need to run), so balancing the accuracy and efficiency is important. To successfully finish a BICePs sampling, lambda values of 0.0 and 1.0 are necessary. Based on our experience, the lambda values of 0.0,0.5,1.0 are good enough for BICePs sampling.

In [17]:
# REQUIRED: specify how many lambdas to sample (more lambdas will provide higher 
# accuracy but slower the whole process, lambda=0.0 and 1.0 are necessary)
lambda_values = [0.0,0.5,1.0]

Print what experimental observables are included in BICePs sampling. This is optional and provides a chance for double check before running BICePs sampling.

In this tutorial, we used both J couplings and NOE (pairwise distances) in BICePs sampling. 

In [18]:
# OPTIONAL but RECOMMENDED: print experimental restraints included (a chance for double check)
res = list_res(data)
print res
  

['J', 'noe']


Anoter key parameter for BICePs set-up is the type of reference potential for each experimental observables. More information of reference potential can be found [here](https://biceps.readthedocs.io/en/latest/theory.html).

Three reference potentials are supported in BICePs: uniform ('uniform'), exponential ('exp'), Gaussian ('gau').  

As we found in previous research, exponential reference potential is useful in most cases. Some higher level task may require more in reference potential selection (e.g [force field parametrization](https://pubs.acs.org/doi/10.1021/acs.jpcb.7b11871)).

(Noet: It will be helpful to print out what is the order of experimental observables included in BICePs sampling as shown above.)

In [19]:
# REQUIRED: specify reference potential to use for each experimental observable
# will be in the same order as the printed observables from (print res)
ref=['uniform','exp']


If you want, you can specify nuisance parameters (uncertainty in experiments) range but it's not required. Our default parameters range is broad enough.
"gamma" is a scaling factor converting distances to NOE intensity in experiments. It is only used when NOE restraint is ued in BICePs sampling.

In [20]:
# OPTIONAL: specify nuisance parameters for each experimnetal observable
# will be in the same order as the printed observables from (print res)
# only specify if you want to narrow down the default range  
uncern=[[0.05,20.0,1.02],[0.05,5.0,1.02]]
gamma = [0.2,5.0,1.01]  

Now we need to set up BICePs samplings and feed arguments using variables we defined above. In most cases, you don't need to modify this part as long as you defined all these parameters shown above. 

In [21]:
######################
# Main:
######################

for j in lambda_values:
    print 'lambda', j
    verbose = False #False
    lam = j
    # We will instantiate a number of Restraint() objects to construct the ensemble
    # experimental data and pre-computed model data are compiled for each state
    ensemble = []
    for i in range(energies.shape[0]):   # number of states
        if verbose:
            print '\n#### STRUCTURE %d ####'%i
        ensemble.append([])
        for k in range(len(data[0])):   # number of experimental observables
            File = data[i][k]
            if verbose:
                print File
            R=init_res('top/%d.fixed.pdb'%i,lam,energies[i],ref[k],File,uncern[k],gamma)
            ensemble[-1].append(R)
        #print ensemble
        
    ##########################################
    # Next, let's do posterior MCMC sampling
    ########## Posterior Sampling ############

    sampler = PosteriorSampler(ensemble)
    sampler.compile_nuisance_parameters()

    sampler.sample(nsteps)  # number of steps

    print 'Processing trajectory...',

    sampler.traj.process()  # compute averages, etc.
    print '...Done.'
    print 'Writing results...',
    sampler.traj.write_results(os.path.join(outdir,'traj_lambda%2.2f.npz'%lam))
    print '...Done.'
    sampler.traj.read_results(os.path.join(outdir,'traj_lambda%2.2f.npz'%lam))     
    print 'Pickling the sampler object ...',
    outfilename = 'sampler_lambda%2.2f.pkl'%lam
    print outfilename,
    fout = open(os.path.join(outdir, outfilename), 'wb')
    # Pickle dictionary using protocol 0.
    cPickle.dump(sampler, fout)
    fout.close()
    print '...Done.'



lambda 0.0

Accepted 78.9713 % 


Accepted [ 24.5301  24.2915  23.8452   6.3045] % 

Processing trajectory... ...Done.
Writing results... ...Done.
Pickling the sampler object ... sampler_lambda0.00.pkl ...Done.
lambda 0.5

Accepted 75.1632 % 


Accepted [ 24.5772  24.337   23.8817   2.3673] % 

Processing trajectory... ...Done.
Writing results... ...Done.
Pickling the sampler object ... sampler_lambda0.50.pkl ...Done.
lambda 1.0

Accepted 73.8704 % 


Accepted [ 24.4841  24.3542  23.8694   1.1627] % 

Processing trajectory... ...Done.
Writing results... ...Done.
Pickling the sampler object ... sampler_lambda1.00.pkl ...Done.


Now you can check your pre-defined output folder and see if the output files from BICePs sampling are there. In this case, we defined `outdir` as 'results_ref_normal'.

In [22]:
ls results_ref_normal/*

results_ref_normal/sampler_lambda0.00.pkl
results_ref_normal/sampler_lambda0.50.pkl
results_ref_normal/sampler_lambda1.00.pkl
results_ref_normal/traj_lambda0.00.npz
results_ref_normal/traj_lambda0.50.npz
results_ref_normal/traj_lambda1.00.npz


Then, we need to run functions in [Analysis](https://biceps.readthedocs.io/en/latest/api.html#analysis) to get predicted populations of each conformational states and BICePs score. A figure of posterior distribution of populations and nuisance parameters will be plotted as well. 

In [23]:
#########################################
# Let's do analysis using MBAR algorithm and plot figures
############ MBAR and Figures ###########
# Specify necessary argument values

A = Analysis(100,dataFiles,outdir)
A.plot()


Loading results_ref_normal/traj_lambda0.00.npz ...
Loading results_ref_normal/traj_lambda0.50.npz ...
Loading results_ref_normal/traj_lambda1.00.npz ...
Loading results_ref_normal/sampler_lambda0.00.pkl ...
Loading results_ref_normal/sampler_lambda0.50.pkl ...
Loading results_ref_normal/sampler_lambda1.00.pkl ...
lam = [0.0, 0.5, 1.0]
nstates 100
Writing BS.dat...

/Users/yunhuige/anaconda/lib/python2.7/site-packages/pymbar-3.0.3-py2.7-macosx-10.6-x86_64.egg/pymbar/mbar.py:919: RuntimeWarning: invalid value encountered in sqrt
  sigma = np.sqrt(covA_ij[0:K,0:K].diagonal())
/Users/yunhuige/anaconda/lib/python2.7/site-packages/matplotlib/scale.py:101: RuntimeWarning: invalid value encountered in less_equal
  a[a <= 0.0] = 1e-300



...Done.
Writing populations.dat...
...Done.


The output files include: population information ("populations.dat"), figure of sampled parameters distribution ("BICePs.pdf"), BICePs score information ("BS.dat").

First, let's take a look at the populations file.

In [24]:
fin = open('populations.dat','r')
text = fin.read()
fin.close()
print text

1.848100802180208113e-02 1.894884993601575474e-03 2.410698459454074793e-05 1.280402543365765722e-03 1.329988784903084144e-04 1.696905651117369616e-06
4.761675616906435593e-03 6.280459740108934596e-04 1.027840908274235687e-05 6.462804530015955982e-04 8.551182227710229903e-05 1.400704430581652832e-06
7.172209802804641043e-03 6.660350849429397257e-03 7.674393477665741869e-04 5.927516519678294182e-04 5.492710384965966914e-04 6.371095205278202707e-05
1.009932844303840227e-03 2.804069185777802176e-04 9.660237118902514908e-06 2.799663067303218966e-04 7.777082748460685947e-05 2.680166694591291964e-06
1.905086325399985014e-04 9.433407857750708558e-06 5.795960333898619865e-08 1.346949549305315760e-04 6.670650712372997174e-06 4.098629525312328793e-08
1.399662387020939036e-03 3.376028756552074128e-07 1.010391770250862464e-11 3.737592638299200797e-04 9.025559100580362788e-08 0.000000000000000000e+00
7.411622035859766200e-03 5.653324273974913439e-03 5.350536901649327604e-04 6.342537688941091763e-04 

There are 100 rows corresponding to 100 clustered states. 6 columns corresponding to populations of each state (row) for 3 lambda values (first 3 columns) and population change (last 3 columns). 

Then let's take a look at the output figure ("BICePs.pdf").

<img src="BICePs.png" height="500" width="500" align="center">

The top left panel shows the population of each states in the presence (y-axis) and absense (x-axis) of computational modeling prior information. You can find some states (e.g state 38, 59) get awarded after including computational prior information. If you check [this work](https://onlinelibrary.wiley.com/doi/full/10.1002/jcc.23738) you will see how misleading the results will be if we only use experimental restraints without computational prior information. 
The other three panels show the distribution of nuisance parameters in the presence or absence of computational modeling information. It may not be clear in this example due to the limit of finite sampling, but based on our experience, including prior information from our simulations will lower the nuisance parameters than only using experimental restraints.

In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../../theme.css", "r").read()
    return HTML(styles)
css_styling()